<a href="https://colab.research.google.com/github/rvaldivia95/CharityML/blob/master/DecisionTree_Charity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
import seaborn as sns

import graphviz
import pydotplus
import io
from scipy import misc

%matplotlib inline

In [0]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle

In [0]:
# Upload kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!pip install kaggle

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"raimundovb","key":"7f8e2800afe7469feb9049847de092fe"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle config path -p /content

usage: kaggle config [-h] {view,set,unset} ...
kaggle config: error: argument command: invalid choice: 'path' (choose from 'view', 'set', 'unset')


In [0]:
!kaggle competitions download -c udacity-mlcharity-competition

  0% 0.00/512k [00:00<?, ?B/s]
100% 512k/512k [00:00<00:00, 70.1MB/s]
  0% 0.00/437k [00:00<?, ?B/s]
100% 437k/437k [00:00<00:00, 61.8MB/s]
  0% 0.00/342k [00:00<?, ?B/s]
100% 342k/342k [00:00<00:00, 104MB/s]


In [0]:
import pandas as pd
data = pd.read_csv('census.csv.zip', compression='zip', header=0, sep=',', quotechar='"')

In [0]:
data.head()

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [0]:
data['income'].replace("<=50K",0, inplace = True)
data['income'].replace(">50K",1, inplace = True)

In [0]:
labels_workclass, uniques_workclass = pd.factorize(data["workclass"])
labels_education_level, uniques_education_level = pd.factorize(data["education_level"])
labels_marital_status, uniques_marital_status = pd.factorize(data["marital-status"])
labels_occupation, uniques_occupation = pd.factorize(data["occupation"])
labels_race , uniques_race = pd.factorize(data["race"])
labels_relationship 	, uniques_relationship = pd.factorize(data["relationship"])
labels_sex 	, uniques_sex 	 = pd.factorize(data["sex"])
labels_native_country, uniques_native_country = pd.factorize(data["native-country"])

In [0]:
print("Numeric Representation : \n", labels_native_country) 
print("Unique Values : \n", uniques_native_country) 
np.shape(labels_native_country)

data["workclass"]= labels_workclass
data['education_level'] = labels_education_level
data['marital-status'] = labels_marital_status
data['occupation'] = labels_occupation
data['race'] = labels_race
data['relationship'] = labels_relationship
data['sex'] = labels_sex
data['native-country'] = labels_native_country

dataf = data.drop(columns=['education-level'])

dataf.head()


Numeric Representation : 
 [0 0 0 ... 0 0 0]
Unique Values : 
 Index([' United-States', ' Cuba', ' Jamaica', ' India', ' Mexico',
       ' Puerto-Rico', ' Honduras', ' England', ' Canada', ' Germany', ' Iran',
       ' Philippines', ' Poland', ' Columbia', ' Cambodia', ' Thailand',
       ' Ecuador', ' Laos', ' Taiwan', ' Haiti', ' Portugal',
       ' Dominican-Republic', ' El-Salvador', ' France', ' Guatemala',
       ' Italy', ' China', ' South', ' Japan', ' Yugoslavia', ' Peru',
       ' Outlying-US(Guam-USVI-etc)', ' Scotland', ' Trinadad&Tobago',
       ' Greece', ' Nicaragua', ' Vietnam', ' Hong', ' Ireland', ' Hungary',
       ' Holand-Netherlands'],
      dtype='object')


,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,0,0,13.0,0,0,0,0,0,2174.0,0.0,40.0,0,0
1,50,1,0,13.0,1,1,1,0,0,0.0,0.0,13.0,0,0
2,38,2,1,9.0,2,2,0,0,0,0.0,0.0,40.0,0,0
3,53,2,2,7.0,1,2,1,1,0,0.0,0.0,40.0,0,0
4,28,2,0,13.0,1,3,2,1,1,0.0,0.0,40.0,1,0


In [0]:
dataf.head(100)

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,0,0,13.0,0,0,0,0,0,2174.0,0.0,40.0,0,0
1,50,1,0,13.0,1,1,1,0,0,0.0,0.0,13.0,0,0
2,38,2,1,9.0,2,2,0,0,0,0.0,0.0,40.0,0,0
3,53,2,2,7.0,1,2,1,1,0,0.0,0.0,40.0,0,0
4,28,2,0,13.0,1,3,2,1,1,0.0,0.0,40.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20,2,5,10.0,0,9,3,0,1,0.0,0.0,40.0,0,0
96,29,2,1,9.0,0,10,0,0,0,0.0,0.0,40.0,0,0
97,32,5,1,9.0,1,10,1,0,0,7688.0,0.0,40.0,0,1
98,30,2,2,7.0,0,2,3,0,0,0.0,0.0,40.0,0,0


In [0]:
train, test = train_test_split(dataf, test_size = 0.15)
print("Training size: {}; Test size: {}".format(len(train), len(test)))

Training size: 38438; Test size: 6784


In [0]:
c = DecisionTreeClassifier(min_samples_split = 100)

In [0]:
features= ["age" ,	"workclass" ,	"education_level" ,	"education-num" ,	"marital-status" ,	"occupation" ,	"relationship" ,	"race" ,	"sex" ,	"capital-gain" ,	"capital-loss" ,	"hours-per-week" ,	"native-country"]

In [0]:
X_train = train[features]
Y_train= train["income"]

X_test = train[features]
Y_test= train["income"]

In [0]:
dt = c.fit(X_train, Y_train)

In [0]:
def show_tree(tree, features, path):
  f = io.StringIO()
  export_graphviz(tree, out_file=f, feature_names=features)
  pydotplus.graph_from_dot_data(f.getvalue()).write_png(path)
  img = plt.imread(path)
  plt.rcParams["figure.figsize"] = [20, 20]
  plt.imshow(img)


In [0]:
show_tree(dt,features,'dec_tree_01.png')

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
27095,45,1,0,13.0,1,10,1,0,0,0.0,0.0,20.0,0
17571,21,2,1,9.0,0,7,3,0,0,0.0,0.0,24.0,0
25544,50,2,2,7.0,2,4,4,0,1,0.0,0.0,40.0,0
15102,38,2,1,9.0,1,9,1,0,0,0.0,0.0,40.0,0
36112,30,2,9,11.0,0,1,3,0,0,0.0,0.0,36.0,0


In [0]:
y_pred = c.predict(X_test)

In [0]:
from sklearn.metrics import accuracy_score

score = accuracy_score(Y_test, y_pred) * 100

In [0]:
print(" Validation Accuracy using Decision Tree: ", round(score, 1), "%")

 Accuracy using Decision Tree:  87.4 %


In [0]:
data_test = pd.read_csv('test_census.csv.zip', compression='zip', header=0, sep=',', quotechar='"')
data_test.head(100)

,Unnamed: 0,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,0,21.0,Private,10th,6.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,40.0,United-States
1,1,49.0,Private,Bachelors,13.0,Married-civ-spouse,Adm-clerical,Wife,White,Female,0.0,0.0,40.0,United-States
2,2,44.0,Self-emp-not-inc,Assoc-acdm,12.0,Married-civ-spouse,Other-service,Wife,White,Female,0.0,0.0,99.0,United-States
3,3,34.0,Private,Bachelors,13.0,Married-civ-spouse,Sales,Husband,White,Male,7298.0,0.0,46.0,United-States
4,4,24.0,Private,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,48.0,Local-gov,Masters,14.0,Separated,Prof-specialty,Not-in-family,White,Male,0.0,1876.0,50.0,United-States
96,96,40.0,Local-gov,Some-college,10.0,Never-married,Exec-managerial,Not-in-family,White,Male,0.0,0.0,40.0,United-States
97,97,32.0,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0.0,0.0,40.0,United-States
98,98,24.0,Private,HS-grad,9.0,Never-married,Adm-clerical,Unmarried,White,Male,0.0,0.0,50.0,United-States


In [0]:
labels_workclass, uniques_workclass = pd.factorize(data_test["workclass"])
labels_education_level, uniques_education_level = pd.factorize(data_test["education_level"])
labels_marital_status, uniques_marital_status = pd.factorize(data_test["marital-status"])
labels_occupation, uniques_occupation = pd.factorize(data_test["occupation"])
labels_race , uniques_race = pd.factorize(data_test["race"])
labels_relationship 	, uniques_relationship = pd.factorize(data_test["relationship"])
labels_sex 	, uniques_sex 	 = pd.factorize(data_test["sex"])
labels_native_country, uniques_native_country = pd.factorize(data_test["native-country"])

data_test["workclass"]= labels_workclass
data_test['education_level'] = labels_education_level
data_test['marital-status'] = labels_marital_status
data_test['occupation'] = labels_occupation
data_test['race'] = labels_race
data_test['relationship'] = labels_relationship
data_test['sex'] = labels_sex
data_test['native-country'] = labels_native_country

data_test.head(100)

,Unnamed: 0,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,0,21.0,0,0,6.0,0,0,0,0,0,0.0,0.0,40.0,0
1,1,49.0,0,1,13.0,0,1,1,0,1,0.0,0.0,40.0,0
2,2,44.0,1,2,12.0,0,2,1,0,1,0.0,0.0,99.0,0
3,3,34.0,0,1,13.0,0,3,0,0,0,7298.0,0.0,46.0,0
4,4,24.0,0,3,9.0,0,4,0,0,0,0.0,0.0,40.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,48.0,2,7,14.0,3,9,3,0,0,0.0,1876.0,50.0,0
96,96,40.0,2,6,10.0,2,8,3,0,0,0.0,0.0,40.0,0
97,97,32.0,0,1,13.0,0,9,0,2,0,0.0,0.0,40.0,0
98,98,24.0,0,3,9.0,2,1,4,0,0,0.0,0.0,50.0,0


In [0]:
data_test.isnull().values.any()
data_test.isnull().sum()
data_testf = data_test.fillna(-1)
data_testf = data_testf.drop(columns="Unnamed: 0")

In [0]:
result = c.predict(data_testf)

In [0]:
real_results = pd.read_csv('example_submission.csv', header=0, sep=',', quotechar='"')
real_results.head()

id = np.arange(0, 45222, 1).tolist()


d = {'id': id, 'income': result}

df = pd.DataFrame(data=d)

df


,id,income
0,0,0
1,1,0
2,2,0
3,3,1
4,4,0
...,...,...
45217,45217,0
45218,45218,0
45219,45219,0
45220,45220,1
